In [1]:
import pandas as pd

df_typed = pd.read_csv('typed_letters_with_entities_final.tsv',sep='\t')
df_handwritten = pd.read_csv('handwritten_letters_with_entities_final.tsv',sep='\t')
df = pd.concat([df_typed,df_handwritten])

In [2]:
df.head()

,part_of.title,images.reference,filename,is_missing,path_to_image,page_type,path_to_text,path_to_handwritten_info,people,handwritten_info
0,"Correspondence, editorial office 'Goed Wonen' ...",..\images\jpg\757\0001246757.jpg,1246757.0,False,/ivi/ilps/projects/ArtDATIS/valkema/MMRKD07_00...,typed,/ivi/ilps/projects/ArtDATIS/artdatis/tagging/O...,NaN,"{'Spier', 'Haanstra', 'Haan'}",NaN
1,"Correspondence, editorial office 'Goed Wonen' ...",..\images\jpg\763\0001246763.jpg,1246763.0,False,/ivi/ilps/projects/ArtDATIS/valkema/MMRKD07_00...,typed,/ivi/ilps/projects/ArtDATIS/artdatis/tagging/O...,NaN,"{'Bert Haanstra', 'E.Spier', 'Valkema', 'I.Val...",NaN
2,"Correspondence, editorial office 'Goed Wonen' ...",..\images\jpg\765\0001246765.jpg,1246765.0,False,/ivi/ilps/projects/ArtDATIS/valkema/MMRKD07_00...,typed,/ivi/ilps/projects/ArtDATIS/artdatis/tagging/O...,NaN,"{'P. A. Stoon', 'W l', 'Hint Cre', 'M. Th. Bru...",NaN
3,"Correspondence, editorial office 'Goed Wonen' ...",..\images\jpg\767\0001246767.jpg,1246767.0,False,/ivi/ilps/projects/ArtDATIS/valkema/MMRKD07_00...,typed,/ivi/ilps/projects/ArtDATIS/artdatis/tagging/O...,NaN,set(),NaN
4,"Correspondence, editorial office 'Goed Wonen' ...",..\images\jpg\769\0001246769.jpg,1246769.0,False,/ivi/ilps/projects/ArtDATIS/valkema/MMRKD07_00...,typed,/ivi/ilps/projects/ArtDATIS/artdatis/tagging/O...,NaN,"{'Goed Wone', 'Wim Crouwel', 'Kho Liang Ile', ...",NaN


In [13]:
all_people_lst = [list(eval(s)) for s in df['people'].tolist() if isinstance(s,str)]
all_people_lst_flat = sum(all_people_lst,[])

In [14]:
import pickle
pickle.dump(set(all_people_lst_flat), open('all_people_in_correspondence_latest.p','wb'))

In [15]:
from collections import Counter

cnt = Counter(p for p in all_people_lst_flat if len(p) > 3).most_common()


In [16]:
popularity_per_name = dict(cnt)
# popularity_per_name

In [59]:
# cnt

In [60]:
popularity_per_name

{'Sybren Valkema': 301,
 'Valkema': 293,
 'Durk': 129,
 'Corning': 121,
 'Gerrit Rietveld': 114,
 'Harvey Littleton': 109,
 'Sybren': 70,
 'Harvey': 68,
 'Richard Meitner': 63,
 'Dale Chihuly': 63,
 'Rietveld': 57,
 'Pilchuck': 56,
 'Anna': 52,
 'Alice Rooney': 52,
 'Veronique': 50,
 'Erwin Eisch': 49,
 'Mieke Groot': 43,
 'Littleton': 43,
 'Stanislav Libensky': 42,
 'Durk Valkema': 38,
 'Marvin Lipofsky': 38,
 'John': 36,
 'Sybre': 36,
 'Valk': 31,
 'Tom Buechner': 31,
 'Véronique': 31,
 'Bertil Vallien': 31,
 'Heesen': 30,
 'Sam Herman': 30,
 'Rietvel': 29,
 'Willem Heesen': 29,
 'Claude Morin': 29,
 'S. Valkema': 28,
 'Thomas S. Buechner': 27,
 'Corning Museum': 27,
 'Copier': 27,
 'Libensky': 26,
 'S.Valkema': 26,
 'Dale': 26,
 'Chairman': 25,
 'Valke': 25,
 'Helmut Ricke': 25,
 'Peter Rath': 24,
 'Yvonne Brunhammer': 24,
 'Valkem': 23,
 'Glass': 23,
 'Dirk': 23,
 'Ann Warff': 22,
 'Gary Beecham': 22,
 'Wilhelminal': 21,
 'Valkena': 21,
 'Wilhelm': 21,
 'Thomas Buechner': 21,
 'Kea

In [10]:
from rapidfuzz import fuzz

In [72]:
import re
tussenvoegsels_original = ['van', 'de', 'der', 'den',
                 'von', '\'t', 'aan', 'ter',
                 'le', 'du']
tussenvoegsels = tussenvoegsels_original+ [tv.capitalize() for tv in tussenvoegsels_original]
tussenvoegsels += [tv.upper() for tv in tussenvoegsels_original]
tussenvoegsels += ['v', 'd'] # these two are always lowercase

def firstname_lastname(name):
    tokens = re.split('[ ,.]',name.strip(',.'))
    tokens = [token for token in tokens if token]
    if len(tokens) == 1:
        return tokens[0], '' # assume this is first name
    if len(tokens) == 0:
        return '',''
    
    lastname = tokens[-1]
    last_token_num = -1
    for token in tokens[-2::-1]: 
        if token in tussenvoegsels:
            lastname = token + ' ' + lastname
            last_token_num -= 1
        else:
            break
            
    firstname = ' '.join(tokens[:last_token_num])
    if firstname.lower() in ['dhr','dear','herr']:
        firstname = ''
    return firstname,lastname

In [73]:
firstname_lastname('D. Valkema')

('D', 'Valkema')

In [74]:
def initial_matches(prefix1, prefix2):
    if len(prefix1) == 0 or len(prefix2) == 0:
        return False
    score = 0
    tokens1 = [token for token in re.split('[ ,.]',prefix1.lower()) if token]
    tokens2 = [token for token in re.split('[ ,.]',prefix2.lower()) if token]
#     print(tokens1)
#     print(tokens2)
    for token1 in tokens1:
        for token2 in tokens2:
            if token1.startswith(token2):
                score += 1
            if token2.startswith(token1):
                score += 1
    score = score / (len(tokens1)*len(tokens2))
#     print(score)
    return score > 0.5

In [75]:
initial_matches('Erwin','E')

True

In [76]:
def safe_fuzzy_ratio(s1, s2):
    """
    Returns a fuzzy matching score between two strings, ensuring that
    comparisons involving empty strings do not return 100%.

    Args:
    - s1 (str): First string to compare.
    - s2 (str): Second string to compare.

    Returns:
    - int: Fuzzy matching score between 0 and 100.
    """
    if not s1 or not s2:
        # If either string is empty, return 50 to avoid a 100% match.
        return 50
    return fuzz.ratio(s1, s2)

In [77]:
def matches(s1, s2):
    prefix1, lastname1 = firstname_lastname(s1)
    prefix2, lastname2 = firstname_lastname(s2)

    lastname_score = 0.3*safe_fuzzy_ratio(lastname1, lastname2)+0.7*safe_fuzzy_ratio(lastname1.lower(),
                                                                        lastname2.lower()) # because tussenvoegsels are lower case, might be informative
    if lastname_score < 50:
        return False, lastname_score, -1, -1,-1 # to speed up
    if prefix1 or prefix2: # at least one not empty
        prefix_score = 100 if initial_matches(prefix1, prefix2) else safe_fuzzy_ratio(prefix1,prefix2) # first names and stuff, shouldn't be empty
    else:
        prefix_score = lastname_score
        
    substring_score = fuzz.partial_ratio(s1, s2) # Durk, Durk Valkema
    
    total_score =  0.55*lastname_score + 0.35*prefix_score + 0.1*substring_score
    
    if total_score > 55: # "Dhr S. Valkema", "S.Valkema" 
        return True, lastname_score, prefix_score, substring_score, total_score
    return False, lastname_score, prefix_score, substring_score, total_score

In [78]:
matches('Durk','Durk Valkema')

(True, 50.0, 100, 100.0, 72.5)

In [69]:
from tqdm.auto import tqdm

In [44]:
popularity_per_name['Arjan Muylwijk']

12

In [45]:
popularity_per_name['Durk']

129

In [46]:
import numpy as np
def split_entity(long_entity):
    # Example: "Sybren Valkema Durk Valkema"
#     print(long_entity)
    if long_entity.count(' ') < 2:
        return [long_entity] if long_entity in popularity_per_name else []
    
    words = long_entity.split()
    cur_entity = words[0]
    cur_popularity = popularity_per_name[cur_entity] if cur_entity in popularity_per_name else 0 
    popularities = [cur_popularity]
    for word in words[1:]:
        potential_new_entity = cur_entity + ' ' + word
        new_popularity = popularity_per_name[potential_new_entity] if potential_new_entity in popularity_per_name else 0
        popularities.append(new_popularity)
        
    last_word_ind = np.argmax(popularities[1:])+1 # we don't want entities that are only one word long
    best_popularity = popularities[last_word_ind]
    
#     print(popularities,popularities[1:],last_word_ind,best_popularity)
    # Keep the original long entity
    if long_entity in popularity_per_name: 
        res_cur_entity = [long_entity]
    else:
        res_cur_entity = []
    
    res_new_entity = [" ".join(words[:last_word_ind+1])] if popularities[last_word_ind] > 0 else []
    remaining_words = " ".join(words[last_word_ind+1:])
    res = list(set(res_cur_entity+res_new_entity))
    return res + split_entity(remaining_words)


In [47]:
split_entity('Mark v. Praagh')

['Mark v. Praagh']

In [48]:
def split_people(people):
    if not people:
        return people
    ppl = eval(people)
    res = []
    for person in ppl:
        split_ppl = split_entity(person)
        res.extend(split_ppl)
    return set(res)

In [49]:
split_people('{\'M. Th. Brusse\'}')

{'Brusse', 'M. Th. Brusse'}

In [50]:
tqdm.pandas()
df['people_split'] = df['people'].progress_map(split_people)

  0%|          | 0/5833 [00:00<?, ?it/s]

In [51]:
df.to_csv('all_lettters_with_entities_final.tsv',sep='\t',index=False)

In [52]:
all_people_lst_new = [list(s) for s in df['people_split'].tolist()]
all_people_lst_flat_new = sum(all_people_lst_new,[])

cnt_new = Counter(p for p in all_people_lst_flat_new if len(p) > 3).most_common()

In [55]:
popularity_per_name_old = popularity_per_name
popularity_per_name = dict(cnt_new)

In [53]:
pickle.dump(set(all_people_lst_flat_new), open('all_people_in_correspondence_final.p','wb'))

In [54]:
popularity_per_name['Veronique Valkema']

8

In [56]:
cnt_new

[('Valkema', 315),
 ('Sybren Valkema', 303),
 ('Durk', 130),
 ('Corning', 122),
 ('Gerrit Rietveld', 114),
 ('Harvey Littleton', 109),
 ('Sybren', 70),
 ('Harvey', 70),
 ('Dale Chihuly', 66),
 ('Richard Meitner', 64),
 ('Rietveld', 61),
 ('Pilchuck', 56),
 ('Anna', 52),
 ('Alice Rooney', 52),
 ('Veronique', 51),
 ('Erwin Eisch', 50),
 ('Littleton', 45),
 ('Mieke Groot', 44),
 ('Stanislav Libensky', 43),
 ('Durk Valkema', 39),
 ('Marvin Lipofsky', 39),
 ('John', 37),
 ('Sybre', 37),
 ('Valk', 33),
 ('Bertil Vallien', 33),
 ('Heesen', 32),
 ('Véronique', 32),
 ('Sam Herman', 31),
 ('Tom Buechner', 31),
 ('Willem Heesen', 30),
 ('Copier', 30),
 ('S. Valkema', 29),
 ('Glass', 29),
 ('Rietvel', 29),
 ('Claude Morin', 29),
 ('Libensky', 28),
 ('Thomas S. Buechner', 27),
 ('Corning Museum', 27),
 ('Dale', 27),
 ('Valkem', 26),
 ('Chairman', 26),
 ('S.Valkema', 26),
 ('Helmut Ricke', 26),
 ('Glas', 25),
 ('Ann Warff', 25),
 ('Valke', 25),
 ('Linden', 24),
 ('Peter Rath', 24),
 ('Blaricum', 24)

In [136]:
import pickle
alias_edges_new = []
i = 0
for name1, count1 in tqdm(cnt_new):
    for name2, count2 in cnt_new[i+1:]:
        res = matches(name1, name2)
        if res[0]:
            alias_edges_new.append({'name1':name1,
                               'name2':name2,
                               'lastname_score':res[1],
                                'prefix_score':res[2],
                                'substring_score': res[3],
                                'total_score':res[4],
                               'commonness1':count1,
                               'commonness2':count2,
                                   'num_words1':name1.count(' ')+1,
                                   'num_words2':name2.count(' ')+1
                                   })
    i+=1
    if i % 100 == 0:
        print(alias_edges_new[-1])
#         pickle.dump(alias_edges, open('/ivi/ilps/personal/vprovat/alias_edges.p','wb'))
    
    
# next step: find best match by score and commonness

  0%|          | 0/20442 [00:00<?, ?it/s]

{'name1': 'John H. Hauberg', 'name2': 'Dich Hubers', 'lastname_score': 76.92307692307692, 'prefix_score': 19.999999999999996, 'substring_score': 63.1578947368421, 'total_score': 55.62348178137652, 'commonness1': 16, 'commonness2': 1, 'num_words1': 3, 'num_words2': 2}
{'name1': 'Jansen', 'name2': 'J. Vele', 'lastname_score': 50.0, 'prefix_score': 100, 'substring_score': 36.36363636363637, 'total_score': 66.13636363636364, 'commonness1': 10, 'commonness2': 1, 'num_words1': 1, 'num_words2': 2}
{'name1': 'G.Huisman', 'name2': 'Graa', 'lastname_score': 50.0, 'prefix_score': 100, 'substring_score': 40.0, 'total_score': 66.5, 'commonness1': 8, 'commonness2': 1, 'num_words1': 1, 'num_words2': 1}
{'name1': 'isch', 'name2': 'Gerrich', 'lastname_score': 50.0, 'prefix_score': 54.54545454545454, 'substring_score': 85.71428571428572, 'total_score': 55.16233766233766, 'commonness1': 6, 'commonness2': 1, 'num_words1': 1, 'num_words2': 1}
{'name1': 'Helly Oestreicher', 'name2': 'Celle', 'lastname_score

{'name1': 'M e. Heemstra', 'name2': 'M. Heiremans', 'lastname_score': 58.82352941176471, 'prefix_score': 100, 'substring_score': 72.72727272727273, 'total_score': 74.62566844919786, 'commonness1': 1, 'commonness2': 1, 'num_words1': 3, 'num_words2': 2}
{'name1': 'Leerdan Glass', 'name2': 'Ceerde', 'lastname_score': 50.0, 'prefix_score': 61.53846153846154, 'substring_score': 72.72727272727273, 'total_score': 56.31118881118881, 'commonness1': 1, 'commonness2': 1, 'num_words1': 2, 'num_words2': 1}
{'name1': 'hans diet', 'name2': 'Johan', 'lastname_score': 50.0, 'prefix_score': 66.66666666666667, 'substring_score': 75.0, 'total_score': 58.333333333333336, 'commonness1': 1, 'commonness2': 1, 'num_words1': 2, 'num_words2': 1}
{'name1': 'Pagaumek', 'name2': 'P. Lare', 'lastname_score': 50.0, 'prefix_score': 100, 'substring_score': 44.44444444444444, 'total_score': 66.94444444444444, 'commonness1': 1, 'commonness2': 1, 'num_words1': 1, 'num_words2': 2}
{'name1': 'Brevnov', 'name2': 'B. de Neeve

{'name1': 'Chris Wilmarth', 'name2': 'Bris', 'lastname_score': 50.0, 'prefix_score': 66.66666666666667, 'substring_score': 75.0, 'total_score': 58.333333333333336, 'commonness1': 1, 'commonness2': 1, 'num_words1': 2, 'num_words2': 1}
{'name1': 'Lino Tagiiapietra', 'name2': 'Libens', 'lastname_score': 50.0, 'prefix_score': 60.0, 'substring_score': 66.66666666666667, 'total_score': 55.16666666666667, 'commonness1': 1, 'commonness2': 1, 'num_words1': 2, 'num_words2': 1}
{'name1': 'Mary Shirley International', 'name2': 'Théâtre National', 'lastname_score': 73.33333333333333, 'prefix_score': 31.57894736842105, 'substring_score': 66.66666666666667, 'total_score': 58.05263157894737, 'commonness1': 1, 'commonness2': 1, 'num_words1': 3, 'num_words2': 2}
{'name1': 'Dyan Vidulich', 'name2': 'Danfo', 'lastname_score': 50.0, 'prefix_score': 66.66666666666667, 'substring_score': 66.66666666666667, 'total_score': 57.5, 'commonness1': 1, 'commonness2': 1, 'num_words1': 2, 'num_words2': 1}
{'name1': 'A

{'name1': 'Lyle Perkins', 'name2': 'Wyle', 'lastname_score': 50.0, 'prefix_score': 75.0, 'substring_score': 75.0, 'total_score': 61.25, 'commonness1': 1, 'commonness2': 1, 'num_words1': 2, 'num_words2': 1}
{'name1': 'Kon. Wilhelminalaan', 'name2': 'Sybren Valkema Wilhelminalaan', 'lastname_score': 100.0, 'prefix_score': 11.764705882352944, 'substring_score': 88.23529411764706, 'total_score': 67.94117647058825, 'commonness1': 1, 'commonness2': 1, 'num_words1': 2, 'num_words2': 3}
{'name1': 'Profzesson', 'name2': 'P. Lare', 'lastname_score': 50.0, 'prefix_score': 100, 'substring_score': 46.15384615384615, 'total_score': 67.11538461538461, 'commonness1': 1, 'commonness2': 1, 'num_words1': 1, 'num_words2': 2}
{'name1': 'P.j. Mulder', 'name2': 'Le Moule', 'lastname_score': 57.14285714285714, 'prefix_score': 50, 'substring_score': 66.66666666666667, 'total_score': 55.5952380952381, 'commonness1': 1, 'commonness2': 1, 'num_words1': 2, 'num_words2': 2}
{'name1': 'Pierre Disder', 'name2': 'Piet

{'name1': 'Thorn', 'name2': 'Th. Limperg', 'lastname_score': 50.0, 'prefix_score': 100, 'substring_score': 57.14285714285714, 'total_score': 68.21428571428571, 'commonness1': 1, 'commonness2': 1, 'num_words1': 1, 'num_words2': 2}
{'name1': 'H. Hilschenz', 'name2': 'Heël', 'lastname_score': 50.0, 'prefix_score': 100, 'substring_score': 50.0, 'total_score': 67.5, 'commonness1': 1, 'commonness2': 1, 'num_words1': 2, 'num_words2': 1}
{'name1': 'Waldemar', 'name2': 'W. Va', 'lastname_score': 50.0, 'prefix_score': 100, 'substring_score': 57.14285714285714, 'total_score': 68.21428571428571, 'commonness1': 1, 'commonness2': 1, 'num_words1': 1, 'num_words2': 2}
{'name1': 'Michéle de Waard', 'name2': 'de Wauri', 'lastname_score': 75.0, 'prefix_score': 50, 'substring_score': 75.0, 'total_score': 66.25, 'commonness1': 1, 'commonness2': 1, 'num_words1': 3, 'num_words2': 2}
{'name1': 'r. Winter Wilt', 'name2': 'Bolhema Wit', 'lastname_score': 85.71428571428572, 'prefix_score': 13.33333333333333, 'su

{'name1': 'Jörvin', 'name2': 'J. Vele', 'lastname_score': 50.0, 'prefix_score': 100, 'substring_score': 28.57142857142857, 'total_score': 65.35714285714286, 'commonness1': 1, 'commonness2': 1, 'num_words1': 1, 'num_words2': 2}
{'name1': 'Ayten Sürü', 'name2': 'Alte', 'lastname_score': 50.0, 'prefix_score': 66.66666666666667, 'substring_score': 75.0, 'total_score': 58.333333333333336, 'commonness1': 1, 'commonness2': 1, 'num_words1': 2, 'num_words2': 1}
{'name1': 'Urine Littleton Alles', 'name2': 'Firmen Walls', 'lastname_score': 74.0, 'prefix_score': 28.57142857142857, 'substring_score': 54.54545454545454, 'total_score': 56.154545454545456, 'commonness1': 1, 'commonness2': 1, 'num_words1': 3, 'num_words2': 2}
{'name1': 'Having inilleay', 'name2': 'Schavin', 'lastname_score': 50.0, 'prefix_score': 61.53846153846154, 'substring_score': 66.66666666666667, 'total_score': 55.705128205128204, 'commonness1': 1, 'commonness2': 1, 'num_words1': 2, 'num_words2': 1}
{'name1': 'go We', 'name2': 'G

In [137]:
df_alias_raw_new = pd.DataFrame.from_records(alias_edges_new)
df_alias_raw_new

,name1,name2,lastname_score,prefix_score,substring_score,total_score,commonness1,commonness2,num_words1,num_words2
0,Valkema,Valk,50.000000,100.000000,100.000000,72.500000,315,33,1,1
1,Valkema,Valkem,50.000000,100.000000,100.000000,72.500000,315,26,1,1
2,Valkema,Valke,50.000000,100.000000,100.000000,72.500000,315,25,1,1
3,Valkema,Valken,50.000000,76.923077,90.909091,63.513986,315,21,1,1
4,Valkema,Valkena,50.000000,85.714286,85.714286,66.071429,315,21,1,1
...,...,...,...,...,...,...,...,...,...,...
930776,Hd Leerdsin.,Stenhy Hd Leerds,85.714286,100.000000,85.714286,90.714286,1,1,2,3
930777,Tucken,Tucken Wiel Graar Prssy,50.000000,100.000000,100.000000,72.500000,1,1,1,4
930778,James Grant Sa,James Grant Saus,66.666667,100.000000,100.000000,81.666667,1,1,3,3
930779,James Grant Saus,James Grant Saus-Toten,57.142857,100.000000,100.000000,76.428571,1,1,3,3


In [138]:
df_alias_raw_new.to_csv('/ivi/ilps/personal/vprovat/alias_raw_FINAL.tsv',sep='\t',index=False)

In [6]:
df_alias_raw_new = pd.read_csv('/ivi/ilps/personal/vprovat/alias_raw_FINAL.tsv',sep='\t')

In [9]:
df_alias = df_alias_raw_new[(df_alias_raw_new.total_score > 79) & (df_alias_raw_new.lastname_score > 40)& (df_alias_raw_new.prefix_score > 50)] 
df_alias.reset_index().drop('index',axis=1).round(2)

,name1,name2,lastname_score,prefix_score,substring_score,total_score,commonness1,commonness2,num_words1,num_words2
0,Sybren Valkema,S. Valkema,100.00,100.0,88.89,98.89,303,29,2,2
1,Sybren Valkema,S.Valkema,100.00,100.0,87.50,98.75,303,26,2,1
2,Sybren Valkema,Sybren Valkena,85.71,100.0,92.86,91.43,303,14,2,2
3,Sybren Valkema,Sybren Valkem,92.31,100.0,100.00,95.77,303,12,2,2
4,Sybren Valkema,Sybren Valk,72.73,100.0,100.00,85.00,303,10,2,2
...,...,...,...,...,...,...,...,...,...,...
14798,Wiebrand Boukkamp,Wiebrand Boukka,85.71,100.0,100.00,92.14,1,1,2,2
14799,Bolhema Witsen,Bolhema Wit,66.67,100.0,100.00,81.67,1,1,2,2
14800,Hd Leerdsin.,Stenhy Hd Leerds,85.71,100.0,85.71,90.71,1,1,2,3
14801,James Grant Sa,James Grant Saus,66.67,100.0,100.00,81.67,1,1,3,3


In [159]:
df_suspicious = df_alias[df_alias['substring_score']<70]
df_suspicious

,name1,name2,lastname_score,prefix_score,substring_score,total_score,commonness1,commonness2,num_words1,num_words2
271,Sybren Valkema,S.Valken,76.923077,100.0,66.666667,83.974359,303,1,2,1
4382,Sam Herman,S. Federman,71.428571,100.0,66.666667,80.952381,31,3,2,2
4727,S. Valkema,Sybren Valk,72.727273,100.0,66.666667,81.666667,29,10,2,2
4877,S. Valkema,Syhren Valk,72.727273,100.0,66.666667,81.666667,29,2,2,2
4919,S. Valkema,. Sybron Valkeno,71.428571,100.0,66.666667,80.952381,29,1,2,3
...,...,...,...,...,...,...,...,...,...,...
925921,Jan van Ey,Jan Willem van (,72.727273,100.0,62.500000,81.250000,1,1,3,4
928114,Sybren Valkening,S. Valkerna,70.588235,100.0,66.666667,80.490196,1,1,2,2
928255,Aleva Adler,A.Adlerova,76.923077,100.0,66.666667,83.974359,1,1,2,1
929221,jaroslava brychtová,Jaroslava Libensky Brychtov,90.588235,100.0,62.500000,91.073529,1,1,2,3


In [140]:
root_names = set(df_alias.name1.tolist())-set(df_alias.name2.tolist())

In [154]:
df_alias_raw_new[df_alias_raw_new['name2']=='The Metropolitan Museum']

,name1,name2,lastname_score,prefix_score,substring_score,total_score,commonness1,commonness2,num_words1,num_words2
6030,Corning Museum,The Metropolitan Museum,100.000000,26.086957,74.074074,71.537842,27,1,2,3
56326,Albert Museum,The Metropolitan Museum,100.000000,27.272727,72.000000,71.745455,6,1,2,3
75027,Metropolitan,The Metropolitan Museum,50.000000,100.000000,100.000000,72.500000,5,1,1,3
97680,Victoria and Albert Museum,The Metropolitan Museum,100.000000,28.571429,60.606061,71.060606,4,1,4,3
144369,Huntington Museum,The Metropolitan Museum,100.000000,30.769231,71.428571,72.912088,3,1,2,3
245295,The Corning Museum,The Metropolitan Museum,100.000000,51.851852,64.516129,79.599761,2,1,3,3
342899,Pilkington Museum,The Metropolitan Museum,100.000000,30.769231,75.862069,73.355438,1,1,2,3
351093,Everson Museum,The Metropolitan Museum,100.000000,34.782609,72.727273,74.446640,1,1,2,3
492650,albert Museum,The Metropolitan Museum,100.000000,27.272727,72.727273,71.818182,1,1,2,3
499936,N.b. Nuseum,The Metropolitan Museum,83.333333,10.526316,66.666667,56.184211,1,1,2,3


In [152]:
matches('Corning Museum','The Metropolitan Museum')

(True, 100.0, 26.086956521739136, 74.07407407407408, 71.53784219001611)

In [143]:
alias_raw = {'Iep Valkema':'Sybren Valkema',
            'Sybren': 'Sybren Valkema',
            'I. Valkema': 'Sybren Valkema',
            'Durk': 'Durk Valkema',
            'Mevrouw Valkema': 'Veronique Valkema',
            'Heer Valkema':'Valkema'}

def find_canonical_form(name):
    df_matches = df_alias[df_alias.name2==name].sort_values(by=['commonness1',
                                                                'total_score',
                                                                'lastname_score',
                                                   'prefix_score'],ascending=False)
    if len(df_matches)==0:
        return name
#     print(df_matches) # todo remove
    best_match = df_matches.iloc[0]['name1']
    while best_match in alias_raw and alias_raw[best_match] != best_match: # todo make sure this doesn't happen
        best_match = alias_raw[best_match]
    return best_match

In [144]:
find_canonical_form('The Metropolitan Museum')

'The Corning Museum'

In [145]:
#% # remove this line if running for the first time
import pickle
from tqdm.auto import tqdm

for cnt in tqdm(range(0,len(df_alias))): 
    row = df_alias.iloc[cnt]
    name2 = row['name2']
    if name2 in alias_raw: # already matched
        continue
    best_name = find_canonical_form(name2)
    if best_name == name2:
        continue
        
    if popularity_per_name[best_name] == popularity_per_name[name2]: # longer name is seen as canonical
        if len(best_name) < len(name2):
            best_name, name2 = name2, best_name
        
    alias_raw[name2]=best_name
    if cnt % 500 == 0:
        print(cnt, name2, best_name)
        pickle.dump(alias_raw, open('alias_raw_FINAL.p','wb'))

  0%|          | 0/14803 [00:00<?, ?it/s]

0 S. Valkema Sybren Valkema
1000 Joop van den bro Joop van den Broek
3000 lieniqu€e Peters Monique Peters
5500 Van ’t Hoff A.p, Van ’t Hoff
6000 De Neeve de Neeve
9500 Matthijs Sterk Nijmegen Matthijs. Sterk
10000 lL. Davis George L. Davis
10500 Van Deel Van Doel
11000 Bonnie Miller Bonnie J. Miller
11500 A.l.j. Van Der Walle A l.j. Van Der Walle
12000 Rbt Schumann Robert Schumann
14500 Jens H. John H


In [146]:
# matches('Harvey Littleton','Maurine Littleton')

In [147]:
alias_raw

{'Iep Valkema': 'Sybren Valkema',
 'Sybren': 'Sybren Valkema',
 'I. Valkema': 'Sybren Valkema',
 'Durk': 'Durk Valkema',
 'Mevrouw Valkema': 'Veronique Valkema',
 'Heer Valkema': 'Valkema',
 'S. Valkema': 'Sybren Valkema',
 'S.Valkema': 'Sybren Valkema',
 'Sybren Valkena': 'Sybren Valkema',
 'Sybren Valkem': 'Sybren Valkema',
 'Sybren Valk': 'Sybren Valkema',
 'S Valkema': 'Sybren Valkema',
 'Sybren Valke': 'Sybren Valkema',
 'Sybren Valken': 'Sybren Valkema',
 'S. Valkena': 'Sybren Valkema',
 'Sybrem Valkema': 'Sybren Valkema',
 'Sybren Valkama': 'Sybren Valkema',
 'sybren valkema': 'Sybren Valkema',
 'S.Valkena': 'Sybren Valkema',
 'bren Valkema': 'Sybren Valkema',
 'sybren valk': 'Sybren Valkema',
 'gybren valkema': 'Sybren Valkema',
 'sybren Valkema': 'Sybren Valkema',
 'Syben Valkema': 'Sybren Valkema',
 'ybren Valkema': 'Sybren Valkema',
 'Sybren Valkema.': 'Sybren Valkema',
 'S. Valhema': 'Sybren Valkema',
 'Sylren Valkema': 'Sybren Valkema',
 'S. Valkenca': 'Sybren Valkema',
 '

In [155]:
aka_raw = {}
for name1, name2 in alias_raw.items():
    if name2 in aka_raw:
        aka_raw[name2].append(name1)
    else:
        aka_raw[name2] = [name1]
        
aka_raw

{'Sybren Valkema': ['Iep Valkema',
  'Sybren',
  'I. Valkema',
  'S. Valkema',
  'S.Valkema',
  'Sybren Valkena',
  'Sybren Valkem',
  'Sybren Valk',
  'S Valkema',
  'Sybren Valke',
  'Sybren Valken',
  'S. Valkena',
  'Sybrem Valkema',
  'Sybren Valkama',
  'sybren valkema',
  'S.Valkena',
  'bren Valkema',
  'sybren valk',
  'gybren valkema',
  'sybren Valkema',
  'Syben Valkema',
  'ybren Valkema',
  'Sybren Valkema.',
  'S. Valhema',
  'Sylren Valkema',
  'S. Valkenca',
  'S Valkena',
  'S.Valken',
  'sybren Balkema',
  'Wv Sybren Valkema',
  'Sybren Vallkema',
  'W Sybren Valkema',
  'syoren Valkema',
  'Syloren Valkema',
  'I Sybren Valkema',
  'Ol Sybren Valkema',
  'gybren Valkema',
  'ybren valkema',
  'Suyboren Valkema',
  'Syloren Valtema',
  'Lieber Sybrem Valkema',
  'S, Valkema',
  'Loo Sybren Valkema',
  'Sybrne Valkema',
  'Durk Valkema Sybren Valkema',
  'Sybren Valkyma',
  'S,Valkema',
  'Mare Sybren Valkema',
  'Vd. Mare Sybren Valkema',
  'S.Valkea',
  'S Valk',
  

In [149]:
aka_raw['Durk Valkema']

['Durk',
 'D.Valkema',
 'Arjan Muylwijk Durk Valkema',
 'Durk Valke',
 'Durk .Vaikema',
 'ure Valkema',
 'Durk Valkyma',
 'Durk Walkema',
 'Lino Tagliapietra Durk Valkema',
 'Richard Meitner Durk Valkema',
 'D. Valkema',
 'vurk Valkema',
 'D Valkema',
 'Dirk Valkema',
 'Dork Valkema',
 'Duck Valke',
 'Turk Valkema',
 'D. Valleima',
 'Dommelen Valke',
 'Deel Valkema',
 'Derle Valhem',
 'Dink Valkema',
 'Denkmal Valkema',
 'Durtz Valken',
 'Eure Valkema']

In [156]:
import json
json.dump(aka_raw, open('also_known_as_final.json','w'),indent=6)

In [158]:
json.dump(alias_raw, open('alias_raw_final.json','w'),indent=6)

In [ ]:
### OLD CODE BEGINS HERE

In [63]:
#% # remove this line if running for the first time
import pickle
alias_edges = []
i = 0
for name1, count1 in tqdm(cnt):
    for name2, count2 in cnt[i+1:]:
        res = matches(name1, name2)
        if res[0]:
            alias_edges.append({'name1':name1,
                               'name2':name2,
                               'lastname_score':res[1],
                                'prefix_score':res[2],
                                'total_score':res[3],
                               'commonness1':count1,
                               'commonness2':count2})
    i+=1
    if i % 300 == 0:
        print(alias_edges[-1])
#         pickle.dump(alias_edges, open('/ivi/ilps/personal/vprovat/alias_edges.p','wb'))
    
# next step: find best match by score and commonness

  0%|          | 0/21666 [00:00<?, ?it/s]

{'name1': 'Maureen Michelson', 'name2': 'Meureen Michelson', 'lastname_score': 100.0, 'prefix_score': 85.71428571428572, 'total_score': 94.28571428571429, 'commonness1': 8, 'commonness2': 1}
{'name1': 'Cal', 'name2': 'Cail', 'lastname_score': 85.71428571428572, 'prefix_score': 85.71428571428572, 'total_score': 85.71428571428572, 'commonness1': 5, 'commonness2': 1}
{'name1': 'B.Janssen', 'name2': 'Barbara Vasse', 'lastname_score': 66.66666666666666, 'prefix_score': 100, 'total_score': 80.0, 'commonness1': 4, 'commonness2': 1}
{'name1': 'Johannes Schreit', 'name2': 'J. Schii', 'lastname_score': 66.66666666666666, 'prefix_score': 100, 'total_score': 80.0, 'commonness1': 3, 'commonness2': 1}
{'name1': 'zen', 'name2': 'tzen', 'lastname_score': 85.71428571428572, 'prefix_score': 85.71428571428572, 'total_score': 85.71428571428572, 'commonness1': 2, 'commonness2': 1}
{'name1': 'Cornell', 'name2': 'Coronel,', 'lastname_score': 85.71428571428572, 'prefix_score': 85.71428571428572, 'total_score'

{'name1': 'H -D.Huttmann J. Zimmermann', 'name2': 'H. Herrmann', 'lastname_score': 66.66666666666666, 'prefix_score': 100, 'total_score': 80.0, 'commonness1': 1, 'commonness2': 1}
{'name1': 'th. limperg', 'name2': 'Th. Limperg', 'lastname_score': 95.71428571428572, 'prefix_score': 100, 'total_score': 97.42857142857143, 'commonness1': 1, 'commonness2': 1}
{'name1': 'Karicke de Leeuw', 'name2': 'Karicke de Leeu', 'lastname_score': 93.33333333333333, 'prefix_score': 100, 'total_score': 96.0, 'commonness1': 1, 'commonness2': 1}
{'name1': 'S. Lazelle', 'name2': 'S. Lazel', 'lastname_score': 83.33333333333334, 'prefix_score': 100, 'total_score': 90.0, 'commonness1': 1, 'commonness2': 1}
{'name1': 'Francfort', 'name2': 'Frankfort', 'lastname_score': 88.88888888888889, 'prefix_score': 88.88888888888889, 'total_score': 88.88888888888889, 'commonness1': 1, 'commonness2': 1}
{'name1': 'Beuen', 'name2': 'Bueno', 'lastname_score': 80.0, 'prefix_score': 80.0, 'total_score': 80.0, 'commonness1': 1, '

In [64]:
#% # remove this line if running for the first time
df_alias_raw = pd.DataFrame.from_records(alias_edges)
df_alias_raw

,name1,name2,lastname_score,prefix_score,total_score,commonness1,commonness2
0,Sybren Valkema,S. Valkema,100.000000,100.000000,100.000000,301,28
1,Sybren Valkema,S.Valkema,100.000000,100.000000,100.000000,301,26
2,Sybren Valkema,Sybren Valkena,85.714286,100.000000,91.428571,301,13
3,Sybren Valkema,Sybren Valkem,92.307692,100.000000,95.384615,301,12
4,Sybren Valkema,Sybren Valk,72.727273,100.000000,83.636364,301,9
...,...,...,...,...,...,...,...
24184,Stenhy Hd Leerds,Hd Leerdsin.,85.714286,100.000000,91.428571,1,1
24185,Prssy,rssy,88.888889,88.888889,88.888889,1,1
24186,Bolhema Wit,Bolhema Witsen,66.666667,100.000000,80.000000,1,1
24187,James Grant Saus,James Grant Sa,66.666667,100.000000,80.000000,1,1


In [65]:
#% # remove this line if running for the first time
df_alias_raw.to_csv('alias_raw_final.tsv',sep='\t')

In [66]:
df_alias_raw=pd.read_csv('alias_raw_final.tsv',sep='\t').drop('Unnamed: 0',axis=1)

In [67]:
df_alias = df_alias_raw[(df_alias_raw.total_score > 70) & (df_alias_raw.lastname_score > 70)] 
df_alias

,name1,name2,lastname_score,prefix_score,total_score,commonness1,commonness2
0,Sybren Valkema,S. Valkema,100.000000,100.000000,100.000000,301,28
1,Sybren Valkema,S.Valkema,100.000000,100.000000,100.000000,301,26
2,Sybren Valkema,Sybren Valkena,85.714286,100.000000,91.428571,301,13
3,Sybren Valkema,Sybren Valkem,92.307692,100.000000,95.384615,301,12
4,Sybren Valkema,Sybren Valk,72.727273,100.000000,83.636364,301,9
...,...,...,...,...,...,...,...
24182,Wiebrand Boukka,Wiebrand Boukkamp,85.714286,100.000000,91.428571,1,1
24183,Marcel v.d. Dop,Marcel v.d. Do,92.307692,100.000000,95.384615,1,1
24184,Stenhy Hd Leerds,Hd Leerdsin.,85.714286,100.000000,91.428571,1,1
24185,Prssy,rssy,88.888889,88.888889,88.888889,1,1


In [ ]:
x

In [68]:
root_names = set(df_alias.name1.tolist())-set(df_alias.name2.tolist())
root_names

{'F. Black Thomas',
 'H.van den Broek',
 'K Fischer',
 'Ricke',
 'J.c. van der Jag',
 'Angela H8lling',
 'Nenne meine Na',
 'Corning',
 'Shmuel Hanagid St',
 'Crbury A',
 'im U',
 'Amale',
 'Olli',
 'Gis',
 'Vandeur',
 'Douve Egberts',
 'Alain Delon',
 'Karsten',
 'Ebel Brug',
 'Sien van Meurs de fa',
 'Wilke Adolfsson',
 'Bert',
 'Bria Rangelau',
 'Gerd Kruft',
 'Patricia M. Baillargeon',
 'Pe Kea e',
 'Shirley Fineblit',
 'Robert Stone',
 'Louis Comfort Tiffany',
 'Ludovico Santilliana Ven',
 'J & L Lobmeyr',
 'East',
 'Hanneke v',
 'Gretchen Boeing',
 'Andreas Nirschl',
 'Fred Abrams',
 'Lynn Grave',
 'Neb',
 'Mark T. Haley',
 'Klinkenberg',
 'Tetterode',
 'Hank Meloy',
 'J. Laqueur-Réves',
 'Die',
 'John Many',
 'Straw',
 'Den Ha',
 'Washington',
 'Claas Weni',
 'anna pavlown',
 'Douwes Dekker',
 'Glenn',
 'Douglass',
 'Isgard',
 'Jacob',
 'k valkema',
 'Floris Meydam',
 'H.g. von Stockhausen',
 'Hannes den Hartog',
 'Karin Stéckle-',
 'Shistey Rosy',
 'I. Moje-Wohlgemut',
 'Han',


In [69]:
# if commonness is the same, use both names separated by /
# eg. Per Lutken / Per Lutkens


# make a connection if all scores are above 70? or lastname score is 100 and prefix score = ?

In [70]:
#% # remove this line if running for the first time
alias_raw = {'Iep Valkema':'Sybren Valkema'}

In [71]:
alias_raw

{'Iep Valkema': 'Sybren Valkema'}

In [72]:
def find_canonical_form(name):
    df_matches = df_alias[df_alias.name2==name]
    if len(df_matches)==0:
        return name
    best_match = df_matches.sort_values(by=['total_score','commonness1','lastname_score',
                                                   'prefix_score'],ascending=False).iloc[0]['name1']
    while best_match in alias_raw and alias_raw[best_match] != best_match: # todo make sure this doesn't happen
        best_match = alias_raw[best_match]
    return best_match

In [73]:
#% # remove this line if running for the first time
import pickle
from tqdm.auto import tqdm

for cnt in tqdm(range(0,len(df_alias))): 
    row = df_alias.iloc[cnt]
    name2 = row['name2']
    if name2 in alias_raw: # already matched
        continue
    best_name = find_canonical_form(name2)
    if best_name == name2:
        continue
        
    if popularity_per_name[best_name] == popularity_per_name[name2]: # longer name is seen as canonical
        if len(best_name) < len(name2):
            best_name, name2 = name2, best_name
        
    alias_raw[name2]=best_name
    if cnt % 500 == 0:
        print(cnt, name2, best_name)
        pickle.dump(alias_raw, open('alias_raw_final_final.p','wb'))

  0%|          | 0/21288 [00:00<?, ?it/s]

0 S. Valkema Sybren Valkema
1000 Rokexklritz Dreisbach Fritz Dreisbach
1500 Jop von den Er Jop von den Eroek
3000 Roman Bartkiwi Roman Bartkiw
3500 Kurt, Walistab Kurt Wallstab
4500 , Schot Schol
5000 Zan Zand
6500 Meomen Mee
7500 Planta Glant
8000 E. Olive Lan Olive Lane
8500 Crabtree Hall Carol Hall
9000 Jan Willem van Jan van
10500 P, Bone Perry Brown
12000 J.Sano J Sano
13000 Stockhausen von Stockhausen
14000 V Li Vsa Lit
19500 A.g. Stolp M.g. Stolp


In [74]:
import pickle

alias_raw = pickle.load(open('alias_raw_final_final.p','rb'))

In [75]:
aka_raw = {}
for name1, name2 in alias_raw.items():
    if name2 in aka_raw:
        aka_raw[name2].append(name1)
    else:
        aka_raw[name2] = [name1]

In [76]:
aka_raw

{'Sybren Valkema': ['Iep Valkema',
  'S. Valkema',
  'S.Valkema',
  'Sybren Valkena',
  'Sybren Valkem',
  'S Valkema',
  'Sybren Valke',
  'Sybren Valken',
  'S. Valkena',
  'Se Valkema',
  'Sybrem Valkema',
  'Sybren Valkama',
  'sybren valkema',
  'S.Valkena',
  'gybren valkema',
  'sybren Valkema',
  'Syben Valkema',
  'ybren Valkema',
  'Sybren Valkema.',
  'S. Valhema',
  'Sylren Valkema',
  'S. Valkenca',
  'S Valkena',
  'S.Valken',
  'sybren Balkema',
  'Wv Sybren Valkema',
  'Sybren Vallkema',
  'W Sybren Valkema',
  'syoren Valkema',
  'Syloren Valkema',
  'I Sybren Valkema',
  'Ol Sybren Valkema',
  'gybren Valkema',
  'bren Valkema',
  'ybren valkema',
  'Suyboren Valkema',
  'Syloren Valtema',
  'Lieber Sybrem Valkema',
  'S, Valkema',
  'Loo Sybren Valkema',
  'Sybrne Valkema',
  'Durk Valkema Sybren Valkema',
  'Sybren Valkyma',
  'S,Valkema',
  'Mare Sybren Valkema',
  'Vd. Mare Sybren Valkema',
  'S.Valkea',
  'Fles Sybren Valkem',
  'Syb Valkena',
  'Sybren Volkema',

In [77]:
import json
json.dump(aka_raw, open('also_known_as_latest.json','w'),indent=6)